In [2]:
from test_mongo import MongoExplorer
mongo = MongoExplorer()

In [2]:
import pandas as pd
def calc_gain_long(arr, buy_threshold, sell_threshold):
    # arr = [VolumeSum, Buy, Sell, EoH]
    arr = arr[(arr[:, 0] >= buy_threshold) | (arr[:, 0] <= sell_threshold) | (arr[:, 3] == 1)]
    if len(arr) < 2:
        return 0
    l = (arr[:, 0] >= buy_threshold) & (arr[:, 3] == 0)
    s = (arr[:, 0] <= sell_threshold) | (arr[:, 3] == 1)
    buy = l & np.roll(s, 1)
    sell = s & np.roll(l, 1)
    return arr[sell, 2].sum() - arr[buy, 1].sum() 

def calc_gain_short(arr, buy_to_cover_threshold, sell_short_threshold):
    arr = arr[(arr[:, 0] >= buy_to_cover_threshold) | (arr[:, 0] <= sell_short_threshold) | (arr[:, 3] == 1)]
    if len(arr) < 2:
        return 0
    s = (arr[:, 0] <= sell_short_threshold) & (arr[:, 3] == 0)
    l = (arr[:, 0] >= buy_to_cover_threshold) | (arr[:, 3] == 1)
    sell = s & np.roll(l, 1)
    buy = l & np.roll(s, 1)
    return arr[sell, 2].sum() - arr[buy, 1].sum() 

In [5]:
import numpy as np
import pandas as pd
def find_thresholds(symbol, dates, hour, buy_thresholds, sell_thresholds):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    collection_3 = f"{symbol}_ts_trade_prices"
    arr = np.empty((0, 7))
    for date in dates:
        start = f'{date}T{hour}:00:00Z'
        end = f'{date}T{hour+1}:00:00Z'
        query_1 = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
        df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query_1)))
        df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query_1)))
        df = pd.merge(df_1, df_2, on="CurTime")
        df["Dif"] = (df["Price"] - df["Last"])
        df = df.rename(columns={"CurTime": "Time"})
        query_3 = {'$and': [{'Time': {'$gte': start}}, {'Time': {'$lt': end}}]}
        df_3 = pd.DataFrame(list(mongo.mongoDB[collection_3].find(query_3)))
        df = pd.merge(df, df_3, on="Time")
        df = df[["Time", "Side", "TotalSize", "Dif", "BUY", "SELL"]]
        df = df[df["BUY"].notna() & df["SELL"].notna()].reset_index(drop=True)
        df["EoH"] = df["Time"] == f'{date}T{hour}:59:50Z'
        arr = np.concatenate((arr, df.to_numpy()), axis=0)

    df = pd.DataFrame(arr, columns=["Time", "Side", "TotalSize", "Dif", "BUY", "SELL", "EoH"])
    df = df.loc[((df["Side"] == "Bid") & (df["Dif"] > 0))
                | ((df["Side"] == "Ask") & (df["Dif"] < 0))
                | df["EoH"]]
    df["VolumeSum"] = df["TotalSize"] * df["Dif"]
    df = df[["Time", "VolumeSum", "BUY", "SELL", "EoH"]]
    df = df.groupby("Time").agg({"VolumeSum": "sum", "BUY": "first", "SELL": "first", "EoH": "first"})
    arr = df.to_numpy().astype('float64')
    long_arr = np.zeros((len(buy_thresholds) * len(sell_thresholds), 3))
    long_arr[:, 0] = np.repeat(buy_thresholds, len(sell_thresholds))
    long_arr[:, 1] = np.tile(sell_thresholds, len(buy_thresholds))
    short_arr = np.zeros((len(buy_thresholds) * len(sell_thresholds), 3))
    short_arr[:, 0] = np.repeat(buy_thresholds, len(sell_thresholds))
    short_arr[:, 1] = np.tile(sell_thresholds, len(buy_thresholds))
    for i in range(len(long_arr)):
        buy_threshold, sell_threshold = long_arr[i, 0], long_arr[i, 1]
        long_arr[i, -1] = calc_gain_long(arr.copy(), buy_threshold, sell_threshold)
        short_arr[i, -1] = calc_gain_short(arr.copy(), buy_threshold, sell_threshold)
    long_df = pd.DataFrame(long_arr, columns = ["Buy Threshold", "Sell Threshold", "Sum"])
    long_df = long_df.round(3)
    long_df.to_csv(f"tradestation_data/{symbol}_{hour}_thresholds_long.csv", index=False, date_format="%Y-%m-%d")
    long_df = long_df.sort_values(by=["Sum"], ascending=False)
    short_df = pd.DataFrame(short_arr, columns = ["Buy Threshold", "Sell Threshold", "Sum"])
    short_df = short_df.round(3)
    short_df.to_csv(f"tradestation_data/{symbol}_{hour}_thresholds_short.csv", index=False, date_format="%Y-%m-%d")
    short_df = short_df.sort_values(by=["Sum"], ascending=False)
    return long_df.iloc[0, 0], long_df.iloc[0, 1], long_df.iloc[0, 2], short_df.iloc[0, 0], short_df.iloc[0, 1], short_df.iloc[0, 2]

# symbol = "TSLA"
# buy_thresholds = (np.arange(500))
# sell_thresholds = (-np.arange(500))
# hour = 13
# days = ["13", "14", "15", "16", "20", "21", "22", "23", "26", "27", "28", "29", "30"]
# dates = [f"2023-06-{day}" for day in days]
# days = ["03", "05", "06", "07", "10", "11", "12", "13", "14", "17", "18"]
# dates = dates + [f"2023-07-{day}" for day in days]
# print(f"{hour}: {find_thresholds(symbol, dates, hour, buy_thresholds, sell_thresholds)}")
# days = ["08", "09", "12", "13", "14", "15", "16", "20", "21", "22", "23", "26", "27", "28", "29", "30"]
# dates = [f"2023-06-{day}" for day in days]
# days = ["03", "05", "06", "07", "10", "11", "12", "13", "14", "17", "18"]
# dates = dates + [f"2023-07-{day}" for day in days]
# for hour in range(14, 20):
#     print(f"{hour}: {find_thresholds(symbol, dates, hour, buy_thresholds, sell_thresholds)}")
symbols = ['NVDA', 'AMZN', 'AAPL', 'GOOG']
thresholds = {}
for symbol in symbols:
    buy = {}
    sell = {}
    buy_to_cover = {}
    sell_short = {}
    buy_thresholds = (np.arange(500))
    sell_thresholds = (-np.arange(500))
    days = ["24", "25", "26", "28", "31"]
    dates = [f"2023-07-{day}" for day in days]
    for hour in range(13, 20):
        b, s, t1, bc, ss, t2 = find_thresholds(symbol, dates, hour, buy_thresholds, sell_thresholds)
        buy[hour] = b
        sell[hour] = s
        buy_to_cover[hour] = bc
        sell_short[hour] = ss
        print(f"{hour}: {b} {s} {t1}   {bc} {ss} {t2}")
    thresholds[symbol] = {"buy": buy, "sell": sell, "buy_to_cover": buy_to_cover, "sell_short": sell_short}
print(thresholds)  

13: 44.0 -440.0 6.12   17.0 -17.0 11.12
14: 21.0 -5.0 5.27   19.0 -8.0 12.27
15: 8.0 -288.0 9.33   44.0 -72.0 0.69
16: 250.0 0.0 0.0   250.0 0.0 5.67
17: 21.0 -240.0 5.56   499.0 -499.0 0.0
18: 10.0 -17.0 8.59   5.0 -17.0 2.25
19: 57.0 -40.0 2.33   54.0 -2.0 5.6
13: 46.0 -76.0 1.22   260.0 -186.0 1.7
14: 4.0 -59.0 0.51   469.0 -52.0 3.15
15: 0.0 -358.0 2.48   22.0 -58.0 0.03
16: 74.0 -33.0 0.09   178.0 -22.0 1.54
17: 6.0 -57.0 0.52   215.0 -40.0 0.22
18: 9.0 -45.0 1.57   11.0 -42.0 0.36
19: 49.0 -85.0 0.29   29.0 -46.0 0.65
13: 28.0 -46.0 2.26   28.0 -45.0 0.98
14: 4.0 -33.0 1.39   16.0 -38.0 0.55
15: 4.0 -32.0 1.73   36.0 -26.0 0.38
16: 45.0 -21.0 0.19   44.0 -22.0 1.1
17: 34.0 -58.0 0.6   31.0 -15.0 0.44
18: 10.0 -36.0 1.24   48.0 -36.0 1.05
19: 22.0 -66.0 0.03   437.0 -8.0 1.28
13: 60.0 -337.0 3.2   24.0 -57.0 1.08
14: 25.0 -33.0 1.52   26.0 -34.0 1.7
15: 27.0 -475.0 1.01   9.0 -95.0 0.06
16: 31.0 -28.0 0.39   31.0 -29.0 0.9
17: 34.0 -40.0 0.34   34.0 -38.0 0.83
18: 0.0 -10.0 0.92  

In [10]:
import numpy as np
import pandas as pd
def test_gain_long(arr, buy_threshold, sell_threshold):
    # arr = [VolumeSum, Buy, Sell, EoH]
    arr = arr[(arr[:, 0] >= buy_threshold) | (arr[:, 0] <= sell_threshold) | (arr[:, 3] == 1)]
    if len(arr) < 1:
        return 0, None
    l = (arr[:, 0] >= buy_threshold) & (arr[:, 3] == 0)
    s = (arr[:, 0] <= sell_threshold) | (arr[:, 3] == 1)
    buy = l & np.roll(s, 1)
    sell = s & np.roll(l, 1)
    gain = np.empty(len(arr))
    gain[sell] = arr[sell, 2]
    gain[buy] = -arr[buy, 1]
    print(arr)
    print(np.hstack((arr, l[:, None], s[:, None], buy[:, None], sell[:, None], gain[:, None])))
    return gain.sum(), pd.DataFrame(np.hstack((arr, l[:, None], s[:, None], buy[:, None], sell[:, None], gain[:, None])))
    

def test_gain_short(arr, buy_to_cover_threshold, sell_short_threshold):
    arr = arr[(arr[:, 0] >= buy_to_cover_threshold) | (arr[:, 0] <= sell_short_threshold) | (arr[:, 3] == 1)]
    if len(arr) < 1:
        return 0, None
    s = (arr[:, 0] <= sell_short_threshold) & (arr[:, 3] == 0)
    l = (arr[:, 0] >= buy_to_cover_threshold) | (arr[:, 3] == 1)
    sell = s & np.roll(l, 1)
    buy = l & np.roll(s, 1)
    gain = np.empty(len(arr))
    gain[sell] = arr[sell, 2]
    gain[buy] = -arr[buy, 1]
    return gain.sum(), pd.DataFrame(np.hstack((arr, l[:, None], s[:, None], buy[:, None], sell[:, None], gain[:, None])))

def test_thresholds(symbol, dates, hour, buy_threshold, sell_threshold, buy_to_cover_threshold, sell_short_threshold):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    collection_3 = f"{symbol}_ts_trade_prices"
    long_gains = []
    short_gains = []
    for date in dates:
        start = f'{date}T{hour}:00:00Z'
        end = f'{date}T{hour+1}:00:00Z'
        query_1 = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
        df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query_1)))
        df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query_1)))
        df = pd.merge(df_1, df_2, on="CurTime")
        df["Dif"] = (df["Price"] - df["Last"])
        df = df.rename(columns={"CurTime": "Time"})
        query_3 = {'$and': [{'Time': {'$gte': start}}, {'Time': {'$lt': end}}]}
        df_3 = pd.DataFrame(list(mongo.mongoDB[collection_3].find(query_3)))
        df = pd.merge(df, df_3, on="Time")
        df = df[["Time", "Side", "TotalSize", "Dif", "BUY", "SELL"]]
        df = df[df["BUY"].notna() & df["SELL"].notna()].reset_index(drop=True)
        df["EoH"] = df["Time"] == f'{date}T{hour}:59:50Z'
        df = df.loc[((df["Side"] == "Bid") & (df["Dif"] > 0))
                    | ((df["Side"] == "Ask") & (df["Dif"] < 0))
                    | df["EoH"]]
        df["VolumeSum"] = df["TotalSize"] * df["Dif"]
        df = df[["Time", "VolumeSum", "BUY", "SELL", "EoH"]]
        df = df.groupby("Time").agg({"VolumeSum": "sum", "BUY": "first", "SELL": "first", "EoH": "first"})
        arr = df.to_numpy().astype('float64')
        long_gain, long_df = test_gain_long(arr, buy_threshold, sell_threshold)
        short_gain, short_df = test_gain_short(arr, buy_to_cover_threshold, sell_short_threshold)
        long_df.to_csv(f"tradestation_data/{symbol}_{hour}_test_long.csv")
        short_df.to_csv(f"tradestation_data/{symbol}_{hour}_test_short.csv")
        long_gains.append(round(long_gain, 3))
        short_gains.append(round(short_gain, 3))
    return long_gains, sum(long_gains), short_gains, sum(short_gains)
    
symbols = ['NVDA', 'AMZN', 'AAPL', 'GOOG']
thresholds = {'TSLA': {'buy': {13: 4.0, 14: 78.0, 15: 2.0, 16: 8.0, 17: 20.0, 18: 6.0, 19: 44.0},
                'sell': {13: -224.0, 14: -43.0, 15: -68.0, 16: -69.0, 17: -47.0, 18: -70.0, 19: -498.0},
                'buy_to_cover': {13: 15.0, 14: 72.0, 15: 0.0, 16: 14.0, 17: 40.0, 18: 32.0, 19: 111.0},
                'sell_short': {13: -360.0, 14: -28.0, 15: -71.0, 16: -69.0, 17: -39.0, 18: -70.0, 19: 0.0}},
            'NVDA': {'buy': {13: 44.0, 14: 21.0, 15: 8.0, 16: 250.0, 17: 21.0, 18: 10.0, 19: 57.0}, 
                'sell': {13: -440.0, 14: -5.0, 15: -288.0, 16: 0.0, 17: -240.0, 18: -17.0, 19: -40.0}, 
                'buy_to_cover': {13: 17.0, 14: 19.0, 15: 44.0, 16: 250.0, 17: 499.0, 18: 5.0, 19: 54.0}, 
                'sell_short': {13: -17.0, 14: -8.0, 15: -72.0, 16: 0.0, 17: -499.0, 18: -17.0, 19: -2.0}}, 
            'AMZN': {'buy': {13: 46.0, 14: 4.0, 15: 0.0, 16: 74.0, 17: 6.0, 18: 9.0, 19: 49.0}, 
                'sell': {13: -76.0, 14: -59.0, 15: -358.0, 16: -33.0, 17: -57.0, 18: -45.0, 19: -85.0}, 
                'buy_to_cover': {13: 260.0, 14: 469.0, 15: 22.0, 16: 178.0, 17: 215.0, 18: 11.0, 19: 29.0}, 
                'sell_short': {13: -186.0, 14: -52.0, 15: -58.0, 16: -22.0, 17: -40.0, 18: -42.0, 19: -46.0}}, 
            'AAPL': {'buy': {13: 28.0, 14: 4.0, 15: 4.0, 16: 45.0, 17: 34.0, 18: 10.0, 19: 22.0}, 
                'sell': {13: -46.0, 14: -33.0, 15: -32.0, 16: -21.0, 17: -58.0, 18: -36.0, 19: -66.0}, 
                'buy_to_cover': {13: 28.0, 14: 16.0, 15: 36.0, 16: 44.0, 17: 31.0, 18: 48.0, 19: 437.0}, 
                'sell_short': {13: -45.0, 14: -38.0, 15: -26.0, 16: -22.0, 17: -15.0, 18: -36.0, 19: -8.0}}, 
            'GOOG': {'buy': {13: 60.0, 14: 25.0, 15: 27.0, 16: 31.0, 17: 34.0, 18: 0.0, 19: 74.0}, 
                'sell': {13: -337.0, 14: -33.0, 15: -475.0, 16: -28.0, 17: -40.0, 18: -10.0, 19: -446.0}, 
                'buy_to_cover': {13: 24.0, 14: 26.0, 15: 9.0, 16: 31.0, 17: 34.0, 18: 0.0, 19: 60.0}, 
                'sell_short': {13: -57.0, 14: -34.0, 15: -95.0, 16: -29.0, 17: -38.0, 18: -10.0, 19: -5.0}}}
# hour = 13
# days = ["13", "14", "15", "16", "20", "21", "22", "23", "26", "27", "28", "29", "30"]
# dates = [f"2023-06-{day}" for day in days]
# days = ["03", "05", "06", "07", "10", "11", "12", "13", "14", "17", "18", "19", "20"]
#dates = dates + [f"2023-07-{day}" for day in days]
# dates = [f"2023-07-{day}" for day in days]
# print(test_thresholds(symbol, dates, hour, buy_thresholds[hour], sell_thresholds[hour], buy_to_cover_thresholds[hour], sell_short_thresholds[hour]))
# days = ["08", "09", "12", "13", "14", "15", "16", "20", "21", "22", "23", "26", "27", "28", "29", "30"]
# dates = [f"2023-06-{day}" for day in days]
# days = ["03", "05", "06", "07", "10", "11", "12", "13", "14", "17", "18", "19", "20"]
#dates = dates + [f"2023-07-{day}" for day in days]
# dates = [f"2023-07-{day}" for day in days]
# for hour in range(14, 20):
#     print(test_thresholds(symbol, dates, hour, buy_thresholds[hour], sell_thresholds[hour], buy_to_cover_thresholds[hour], sell_short_thresholds[hour]))
days = ["24", "25", "26", "28", "31"]
dates = [f"2023-07-{day}" for day in days]
for symbol in symbols:    
    for hour in range(13, 20):
        print(test_thresholds(symbol, dates, hour, thresholds[symbol]['buy'][hour], thresholds[symbol]['sell'][hour], thresholds[symbol]['buy_to_cover'][hour], thresholds[symbol]['sell_short'][hour]))

[[8.45866e+03 4.43190e+02 4.42920e+02 1.00000e+00]]
[[8.45866e+03 4.43190e+02 4.42920e+02 1.00000e+00 0.00000e+00 1.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00]]
[[ 4.84600e+01  4.55210e+02  4.54910e+02  0.00000e+00]
 [-2.67408e+03  4.59880e+02  4.59770e+02  1.00000e+00]]
[[ 4.84600e+01  4.55210e+02  4.54910e+02  0.00000e+00  1.00000e+00
   0.00000e+00  1.00000e+00  0.00000e+00 -4.55210e+02]
 [-2.67408e+03  4.59880e+02  4.59770e+02  1.00000e+00  0.00000e+00
   1.00000e+00  0.00000e+00  1.00000e+00  4.59770e+02]]
[[5.94400e+01 4.53320e+02 4.52990e+02 0.00000e+00]
 [7.78092e+03 4.54970e+02 4.54890e+02 1.00000e+00]]
[[ 5.94400e+01  4.53320e+02  4.52990e+02  0.00000e+00  1.00000e+00
   0.00000e+00  1.00000e+00  0.00000e+00 -4.53320e+02]
 [ 7.78092e+03  4.54970e+02  4.54890e+02  1.00000e+00  0.00000e+00
   1.00000e+00  0.00000e+00  1.00000e+00  4.54890e+02]]
[[2.2200000e+02 4.6490000e+02 4.6459000e+02 0.0000000e+00]
 [1.0400000e+02 4.6448000e+02 4.6420000e+02 0.0000000e+00]
 [1.2325481e

KeyboardInterrupt: 

In [29]:
def calculate_spread(symbol):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_ts_trade_prices"
    collection_3 = f"{symbol}_ts_trades"
    df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find({})))
    df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find({})))
    df_3 = pd.DataFrame(list(mongo.mongoDB[collection_3].find({})))
    df_1 = df_1.rename(columns={"CurTime": "Time"})
    df = pd.merge(df_1, df_2, on="Time")
    df = pd.merge(df, df_3, on="Time")
    df = df[(df["Action"].notna()) & (df["Price"].notna()) & (df["Last_x"].notna()) & (df["BUY"].notna()) & (df["SELL"].notna())]
    df.to_csv("tradestation_data/TSLA_test.csv")
    buys = (df["Action"] == "BUY") | (df["Action"] == "BUYTOCOVER")
    sells = (df["Action"] == "SELL") | (df["Action"] == "SELLSHORT")
    df.loc[buys, "Buy Spread"] = df.loc[buys, "Price"]/df.loc[buys, "Last_x"]
    df.loc[sells, "Sell Spread"] = df.loc[sells, "Price"]/df.loc[sells, "Last_x"]
    df.loc[buys, "Est Buy Spread"] = df.loc[buys, "BUY"]/df.loc[buys, "Last_x"]
    df.loc[sells, "Est Sell Spread"] = df.loc[sells, "SELL"]/df.loc[sells, "Last_x"]
    df.loc[buys, "Est Buy Dif"] = df.loc[buys, "Price"]/df.loc[buys, "BUY"]
    df.loc[sells, "Est Sell Dif"] = df.loc[sells, "Price"]/df.loc[sells, "SELL"]
    return df["Buy Spread"].mean(), df["Sell Spread"].mean(), df["Est Buy Spread"].mean(), df["Est Sell Spread"].mean(), df["Est Buy Dif"].mean(), df["Est Sell Dif"].mean()

symbol = "TSLA"
calculate_spread(symbol)

(1.000364939838234,
 0.9997002136186819,
 1.000359992850771,
 0.9996909339770573,
 1.0000049420287902,
 1.0000092902472637)

In [12]:
collection = f"SMCI_10sec_ts_lvl1"
df = pd.DataFrame(list(mongo.mongoDB[collection].find({})))
df.tail(100)

,_id,CurTime,GetTime,Symbol,TradeTime,Last,Ask,AskSize,Bid,BidSize,Volume,VWAP,Open,High,Low,PreviousClose,PreviousVolume,NetChange,NetChangePct
208600,64c95fd6ca2cfc9bbc781656,2023-08-01T19:41:10Z,2023-08-01T19:41:08Z,SMCI,2023-08-01T19:40:48Z,336.6398,336.63,100.0,336.32,100.0,1847557.0,332.800750,322.48999,337.30869,320.0,330.26999,2356512.0,6.36981,0.0193
208601,64c95fe0ca2cfc9bbc78171a,2023-08-01T19:41:20Z,2023-08-01T19:41:19Z,SMCI,2023-08-01T19:41:13Z,336.4000,336.62,400.0,336.32,100.0,1848495.0,332.802658,322.48999,337.30869,320.0,330.26999,2356512.0,6.13001,0.0186
208602,64c95feaca2cfc9bbc7817da,2023-08-01T19:41:30Z,2023-08-01T19:41:29Z,SMCI,2023-08-01T19:41:13Z,336.4000,336.62,300.0,336.33,200.0,1848529.0,332.802658,322.48999,337.30869,320.0,330.26999,2356512.0,6.13001,0.0186
208603,64c95ff4ca2cfc9bbc7818a0,2023-08-01T19:41:40Z,2023-08-01T19:41:38Z,SMCI,2023-08-01T19:41:32Z,336.4134,336.60,400.0,336.32,200.0,1849384.0,332.803035,322.48999,337.30869,320.0,330.26999,2356512.0,6.14341,0.0186
208604,64c95ffeca2cfc9bbc781967,2023-08-01T19:41:50Z,2023-08-01T19:41:49Z,SMCI,2023-08-01T19:41:49Z,336.4010,336.58,300.0,336.40,100.0,1850165.0,332.804459,322.48999,337.30869,320.0,330.26999,2356512.0,6.13101,0.0186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208695,64c9638cca2cfc9bbc7860ea,2023-08-01T19:57:00Z,2023-08-01T19:56:58Z,SMCI,2023-08-01T19:56:53Z,337.4199,337.40,200.0,337.14,100.0,1950957.0,332.979160,322.48999,337.41989,320.0,330.26999,2356512.0,7.14991,0.0216
208696,64c96396ca2cfc9bbc7861b6,2023-08-01T19:57:10Z,2023-08-01T19:57:09Z,SMCI,2023-08-01T19:57:07Z,337.3045,337.27,100.0,337.14,600.0,1952465.0,332.981913,322.48999,337.41989,320.0,330.26999,2356512.0,7.03451,0.0213
208697,64c963a0ca2cfc9bbc786277,2023-08-01T19:57:20Z,2023-08-01T19:57:19Z,SMCI,2023-08-01T19:57:15Z,337.2100,337.36,400.0,337.14,300.0,1959566.0,332.997210,322.48999,337.42999,320.0,330.26999,2356512.0,6.94001,0.0210
208698,64c963aaca2cfc9bbc78633a,2023-08-01T19:57:30Z,2023-08-01T19:57:29Z,SMCI,2023-08-01T19:57:27Z,337.3000,337.29,400.0,337.15,200.0,1960187.0,332.997896,322.48999,337.42999,320.0,330.26999,2356512.0,7.03001,0.0213


In [7]:
import pymongo
mongo_client = pymongo.MongoClient('mongodb://localhost:27017/')
mongoDB = mongo_client['stock']
list(mongoDB["test"].find())

[{'_id': ObjectId('64d97502e6ac3cc73cac0aaf'), 'fieldName': 'hello2'}]